# **Data Cleaning**

### 1. Accessing the Database

In [1]:
# Import  data
import sqlalchemy as db
from sqlalchemy import create_engine
import os 
from dotenv import load_dotenv 

# Manipulating the data
import pandas as pd
import warnings
import json

# Plotting and Visualization
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

warnings.filterwarnings(action = 'ignore')


In [2]:
# Access the Database 
# Load and get credentials from the environment 
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")
db_name_raw = os.getenv("DB_RAW")

# Connection string
connection_string = f"mysql+mysqlconnector://{db_user}:{db_password}@{db_host}:3306/db_olist_raw"

# Create SQLAlchemy engine 
try:
    engine = db.create_engine(connection_string)
    print(f"Connection to 'db_olist_raw' successful.")
except Exception as e: 
    print(f"Connection failed: {e}")

# Create a connection to the Database
conn = engine.connect()

# Test query 
test_query = "SELECT * FROM customers_dataset LIMIT 5;"
df_test = pd.read_sql(test_query, conn)
        
print("Test query successful:")
display(df_test)

Connection to 'db_olist_raw' successful.
Test query successful:


,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,00012a2ce6f8dcda20d059ce98491703,248ffe10d632bebe4f7267f1f44844c9,41016,Covington,Kentucky\r
1,000161a058600d5901f007fab4c27140,b0015e09bb4b6e47c52844fab5fb6638,10546,Millwood,New York\r
2,0001fd6190edaaf884bcaf3d49edf079,94b11d37cd61cb2994a194d11f89682b,62279,Renault,Illinois\r
3,0002414f95344307404f0ace7a26f1d5,4893ad4ea28b2c5b3ddf4e82e79db9e6,04742,Fort Fairfield,Maine\r
4,000379cdec625522490c315e70c7a9fb,0b83f73b19c2019e182fd552c048a22c,41016,Covington,Kentucky\r


In [3]:
# Get the list of tables from the database 
with engine.connect() as conn: 
    table_names_query = "SHOW TABLES;"
    table_names = pd.read_sql(table_names_query, conn).iloc[:,0].tolist()
print(f"Found {len(table_names)} tables: {table_names}")

Found 9 tables: ['customers_dataset', 'geolocation_dataset', 'order_items_dataset', 'order_payments_dataset', 'order_reviews_dataset', 'orders_dataset', 'product_category_name_translation', 'products_dataset', 'sellers_dataset']


In [4]:
# Loop through the list of table names and load them into a dictionary 
table_dict = {}
with engine.connect() as conn:
    for table in table_names:
        query = f"SELECT * FROM {table};"
        table_dict[table] = pd.read_sql(query,conn)
print(f"All table from {db_name_raw} database have been loaded into the 'table_dict' dictionary")

All table from db_olist_raw database have been loaded into the 'table_dict' dictionary


In [5]:
# Assign the dataframes to variables 
customers_df = table_dict['customers_dataset']
geolocation_df = table_dict['geolocation_dataset']
order_items_df = table_dict['order_items_dataset']
order_payment_df = table_dict['order_payments_dataset']
order_review_df = table_dict['order_payments_dataset']
orders_df = table_dict['orders_dataset']
product_name_eng = table_dict['product_category_name_translation']
product_df = table_dict['products_dataset']
sellers_df = table_dict['sellers_dataset'] 

### 2. Examine the datasets 

In [6]:
# Create a function to examine the data
def examine_data(df):
    """ Function to examine the data and summarize the results in a table format."""
    total_rows = len(df)
    
    # Handle empty DataFrame case
    if total_rows == 0:
        return pd.DataFrame({
            'rows_count': [0],
            'rows_with_nulls': [0],
            'unique' : [False],
            'cardinality': [0],
            'with_nulls': [False],
            'null_%': [0],
            '1st_row': [None],
            'last_row': [None],
            'random_row': [None],
            'data_type': [None]
        })

    result = pd.DataFrame({
        'rows_count': [total_rows] * df.shape[1],
        'rows_with_nulls': df.isnull().sum(),
        'unique' : df.nunique() == total_rows,
        'cardinality': df.nunique(),
        'with_nulls': df.isnull().any(),
        'null_%': round((df.isnull().sum()/total_rows * 100),1),
        '1st_row': df.iloc[0],
        'last_row': df.iloc[-1],
        'random_row': df.sample(1).iloc[0],
        'data_type': df.dtypes
    })

    return result

In [ ]:
# 1. Examine the "customers table"
examine_customers_dataset = examine_data(customers_df)
display(examine_customers_dataset)
display(customers_df.head(5))

,rows_count,rows_with_nulls,unique,cardinality,with_nulls,null_%,1st_row,last_row,random_row,data_type
customer_id,99441,0,True,99441,False,0.0,00012a2ce6f8dcda20d059ce98491703,ffffe8b65bbe3087b653a978c870db99,cd99a4cb6f4d2e8342351bc978460e07,object
customer_unique_id,99441,0,False,96096,False,0.0,248ffe10d632bebe4f7267f1f44844c9,736e6bfa0510aa5b878881a226a5fd89,eb295bd972e6d73251348a08a6e18ccd,object
customer_zip_code_prefix,99441,0,False,5841,False,0.0,41016,85305,64012,object
customer_city,99441,0,False,3781,False,0.0,Covington,Glendale,Belton,object
customer_state,99441,0,False,25,False,0.0,Kentucky\r,Arizona\r,Missouri\r,object


,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,00012a2ce6f8dcda20d059ce98491703,248ffe10d632bebe4f7267f1f44844c9,41016,Covington,Kentucky\r
1,000161a058600d5901f007fab4c27140,b0015e09bb4b6e47c52844fab5fb6638,10546,Millwood,New York\r
2,0001fd6190edaaf884bcaf3d49edf079,94b11d37cd61cb2994a194d11f89682b,62279,Renault,Illinois\r
3,0002414f95344307404f0ace7a26f1d5,4893ad4ea28b2c5b3ddf4e82e79db9e6,04742,Fort Fairfield,Maine\r
4,000379cdec625522490c315e70c7a9fb,0b83f73b19c2019e182fd552c048a22c,41016,Covington,Kentucky\r


In [ ]:
# 2. Examine the "orders table"
examine_orders_dataset = examine_data(orders_df)
display(examine_orders_dataset)
display(orders_df.head(5))

,rows_count,rows_with_nulls,unique,cardinality,with_nulls,null_%,1st_row,last_row,random_row,data_type
order_id,99441,0,True,99441,False,0.0,e481f51cbdc54678b7cc49136f2d6af7,66dea50a8b16d9b4dee7af250b4be1a5,1299a512fc464f808332d44a0e73f367,object
customer_id,99441,0,True,99441,False,0.0,9ef432eb6251297304e76186b10a928d,edb027a75a1449115f6b43211ae02a24,03607aa94cc886c288008326808df435,object
order_status,99441,0,False,8,False,0.0,delivered,delivered,delivered,object
order_purchase_timestamp,99441,0,False,98875,False,0.0,2017-10-02 10:56:33,2018-03-08 20:57:30,2017-03-19 15:21:25,datetime64[ns]
order_approved_at,99441,160,False,90733,True,0.2,2017-10-02 11:07:15,2018-03-09 11:20:28,2017-03-19 15:21:25,datetime64[ns]
order_delivered_carrier_date,99441,1783,False,81018,True,1.8,2017-10-04 19:55:00,2018-03-09 22:11:59,2017-03-24 13:38:41,datetime64[ns]
order_delivered_customer_date,99441,2965,False,95664,True,3.0,2017-10-10 21:25:13,2018-03-16 13:08:30,2017-04-01 10:53:57,datetime64[ns]
order_estimated_delivery_date,99441,0,False,459,False,0.0,2017-10-18 00:00:00,2018-04-03 00:00:00,2017-04-20 00:00:00,datetime64[ns]


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26


In [ ]:
# 3. Examine the "geolocation table"
examine_geo_dataset = examine_data(geolocation_df)
display(examine_geo_dataset)
display(geolocation_df.head(5))

,rows_count,rows_with_nulls,unique,cardinality,with_nulls,null_%,1st_row,last_row,random_row,data_type
geolocation_zip_code_prefix,2000326,0,False,6498,False,0.0,03304,84326,97039,object
geolocation_lat,2000326,0,False,1000163,False,0.0,43.129723,41.674383,45.475542,float64
geolocation_lng,2000326,0,False,1000163,False,0.0,-71.527521,-111.816504,-120.620244,float64
geolocation_city,2000326,0,False,4120,False,0.0,Bow,Millville,Moro,object
geolocation_state,2000326,0,False,26,False,0.0,New Hampshire\r,Utah\r,Oregon\r,object


,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,03304,43.129723,-71.527521,Bow,New Hampshire\r
1,53190,42.809631,-88.727142,Whitewater,Wisconsin\r
2,14028,43.314741,-78.717849,Burt,New York\r
3,70534,30.162003,-92.454649,Estherwood,Louisiana\r
4,64752,38.072689,-94.568662,Hume,Missouri\r


In [ ]:
# 4. Examine the "order item table"
examine_order_item_dataset = examine_data(order_items_df)
display(examine_order_item_dataset)
display(order_items_df.head(5))

,rows_count,rows_with_nulls,unique,cardinality,with_nulls,null_%,1st_row,last_row,random_row,data_type
order_id,225300,0,False,98666,False,0.0,"""00010242fe8c5a6d1ba2dd792cb16214""",fffe41c64501cc87c801fd61db3f6244,"""442693e8e705ad1f9c91f762f0d297bc""",object
order_item_id,225300,0,False,21,False,0.0,1,1,1,int64
product_id,225300,0,False,32951,False,0.0,"""4244733e06e7ecb4970a6e2683c13e61""","""350688d9dc1e75ff97be326363655e01""",bc4cd4da98dd128c39bf0b8c2674032f,object
seller_id,225300,0,False,3095,False,0.0,"""48436dade18ac8b2bce089ec2a041202""",f7ccf836d21b2fb1de37564105216cc1,"""53243585a1d6dc2643021fd1853d8905""",object
shipping_limit_date,225300,0,False,93318,False,0.0,2017-09-19 09:45:35,2018-06-12 17:10:13,2017-09-03 21:25:14,datetime64[ns]
price,225300,0,False,5968,False,0.0,58.9,43.0,1200.0,float64
freight_value,225300,0,False,6999,False,0.0,13.29,12.79,50.81,float64


,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,"""00010242fe8c5a6d1ba2dd792cb16214""",1,"""4244733e06e7ecb4970a6e2683c13e61""","""48436dade18ac8b2bce089ec2a041202""",2017-09-19 09:45:35,58.90,13.29
1,"""00018f77f2f0320c557190d7a144bdd3""",1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,"""000229ec398224ef6ca0657da4fc703e""",1,c777355d18b72b67abbeef9df44fd0fd,"""5b51032eddd242adc84c38acab88f23d""",2018-01-18 14:48:30,199.00,17.87
3,"""00024acbcdf0a6daa1e931b038114c75""",1,"""7634da152a4610f1595efa32f14722fc""","""9d7a1d34a5052409006425275ba1c2b4""",2018-08-15 10:10:18,12.99,12.79
4,"""00042b26cf59d7ce69dfabb4e55b4fd9""",1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [ ]:
# 5. Examine the "seller table"
examine_sellers_dataset = examine_data(sellers_df)
display(examine_sellers_dataset)
display(sellers_df.head(5))

,rows_count,rows_with_nulls,unique,cardinality,with_nulls,null_%,1st_row,last_row,random_row,data_type
seller_id,3095,0,True,3095,False,0.0,0015a82c2db000af6aaaf3ae2ecb0532,ffff564a4f9085cd26170f4732393726,6c9875b2f94ba781186f0c1aed8d1687,object
seller_zip_code_prefix,3095,0,False,1914,False,0.0,28732,10566,40014,object
seller_city,3095,0,False,1470,False,0.0,Fletcher,Peekskill,Crestwood,object
seller_state,3095,0,False,24,False,0.0,North Carolina\r,New York\r,Kentucky\r,object


,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,0015a82c2db000af6aaaf3ae2ecb0532,28732,Fletcher,North Carolina\r
1,001cca7ae9ae17fb1caed9dfb1094831,70341,Belle Rose,Louisiana\r
2,001e6ad469a905060d959994f1b41e4f,60649,Chicago,Illinois\r
3,002100f778ceb8431b7a1020ff7ab48f,61072,Rockton,Illinois\r
4,003554e2dce176b5555353e4f3555ac8,57319,Bridgewater,South Dakota\r


In [ ]:
# 6. Examine the "order payment table"
examine_order_payment_dataset = examine_data(order_payment_df)
display(examine_order_payment_dataset)
display(order_payment_df.head(5))

,rows_count,rows_with_nulls,unique,cardinality,with_nulls,null_%,1st_row,last_row,random_row,data_type
order_id,207772,0,False,99440,False,0.0,b81ef226f3fe1789b1e8b2acac839d17,"""28bbae6599b09d39ca406b747b6632b1""",e3234cd6072b07fce570475325d2ca5f,object
payment_sequential,207772,0,False,29,False,0.0,1,1,1,int64
payment_type,207772,0,False,5,False,0.0,credit_card,boleto,boleto,object
payment_installments,207772,0,False,24,False,0.0,8,1,1,int64
payment_value,207772,0,False,29077,False,0.0,99.33,191.58,129.57,float64


,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,"""25e8ea4e93396b6fa0d3dd708e76c1bd""",1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,"""42fdf880ba16b47b59251dd489d4441a""",1,credit_card,2,128.45


In [13]:
# 7. Examine the "order review table"
examine_order_review_dataset = examine_data(order_review_df)
display(examine_order_review_dataset)
display(order_review_df.head(5))

,rows_count,rows_with_nulls,unique,cardinality,with_nulls,null_%,1st_row,last_row,random_row,data_type
order_id,207772,0,False,99440,False,0.0,b81ef226f3fe1789b1e8b2acac839d17,"""28bbae6599b09d39ca406b747b6632b1""","""1fffb8b3f671b92497da76aacd6abc72""",object
payment_sequential,207772,0,False,29,False,0.0,1,1,1,int64
payment_type,207772,0,False,5,False,0.0,credit_card,boleto,credit_card,object
payment_installments,207772,0,False,24,False,0.0,8,1,3,int64
payment_value,207772,0,False,29077,False,0.0,99.33,191.58,182.85,float64


,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,"""25e8ea4e93396b6fa0d3dd708e76c1bd""",1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,"""42fdf880ba16b47b59251dd489d4441a""",1,credit_card,2,128.45


In [14]:
# 8. Examine the "order product table"
examine_order_item_dataset = examine_data(product_df)
display(examine_order_item_dataset)
display(product_df.head(5))

,rows_count,rows_with_nulls,unique,cardinality,with_nulls,null_%,1st_row,last_row,random_row,data_type
product_id,32951,0,True,32951,False,0.0,00066f42aeeb9f3007548bb9d3f33c38,fffe9eeff12fcbd74a2f2b007dde0c58,97ac99fefa9cf4eb8f825136a89e8f40,object
product_category_name,32951,0,False,74,False,0.0,perfumaria,brinquedos,telefonia,object
product_name_lenght,32951,610,False,66,True,1.9,53.0,57.0,63.0,float64
product_description_lenght,32951,610,False,2960,True,1.9,596.0,1536.0,3883.0,float64
product_photos_qty,32951,610,False,19,True,1.9,6.0,3.0,1.0,float64
product_weight_g,32951,2,False,2204,True,0.0,300.0,3900.0,370.0,float64
product_length_cm,32951,2,False,99,True,0.0,20.0,43.0,17.0,float64
product_height_cm,32951,2,False,102,True,0.0,16.0,16.0,6.0,float64
product_width_cm,32951,2,False,95,True,0.0,16.0,11.0,11.0,float64


,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,00066f42aeeb9f3007548bb9d3f33c38,perfumaria,53.0,596.0,6.0,300.0,20.0,16.0,16.0
1,00088930e925c41fd95ebfe695fd2655,automotivo,56.0,752.0,4.0,1225.0,55.0,10.0,26.0
2,0009406fd7479715e4bef61dd91f2462,cama_mesa_banho,50.0,266.0,2.0,300.0,45.0,15.0,35.0
3,000b8f95fcb9e0096488278317764d19,utilidades_domesticas,25.0,364.0,3.0,550.0,19.0,24.0,12.0
4,000d9be29b5207b54e86aa1b1ac54872,relogios_presentes,48.0,613.0,4.0,250.0,22.0,11.0,15.0


In [ ]:
# 9. Examine the "english product name table"
display(product_name_eng.head(5))

,product_category_name,product_category_name_english
0,agro_industria_e_comercio,agro_industry_and_commerce\r
1,alimentos,food\r
2,alimentos_bebidas,food_drink\r
3,artes,art\r
4,artes_e_artesanato,arts_and_craftmanship\r


### 3. Data Cleaning 

In [16]:
# 1. Clean the product table and metadata
# Remove "\r" from the "product_category_name_english" strings values 
product_name_eng['product_category_name_english'] = product_name_eng['product_category_name_english'].str.replace('\r','',regex=False)

In [ ]:
# Merge english product name to product tables 
product_cleaned_df = product_df.copy()
product_cleaned_df = pd.mearge